In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.5
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import random
import numba
import statsmodels.api as sm
import glob
import pandas as pd
from sklearn.metrics import r2_score

In [ ]:
files = sorted(glob.glob('/hurrell-scratch2/ivyglade/pdo/storm_reports/StormEvents_details-ftp_v1.0_d*.csv'))

In [ ]:
# Import a single ERA5 file so that we have a lat/lon grid for reference
era5_data = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1940s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19400101-19491231.nc')

In [ ]:
# Subset a CONUS box
era5_data_conus = era5_data['CAPE'].sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1980, 2010)

nino_34 = pdo_functions.calc_nino_34_timeseries(sst, False, 1980, 2010)

In [ ]:
# Standardize nino3.4
nino_34_std = (nino_34 - nino_34.mean()) / nino_34.std()

In [ ]:
# Convert pdo to xarray
pdo_xr = xr.DataArray(pdo, coords={'time':nino_34['time']}, dims=['time'])

# Subset only 1980-2024
pdo_1980_2024 = pdo_xr.sel(time=pdo_xr.time.dt.year.isin(np.arange(1980, 2025, 1)))
nino_34_1980_2024 = nino_34_std.sel(time=nino_34_std.time.dt.year.isin(np.arange(1980, 2025, 1)))

# only MAM
pdo_1980_2024_mam = pdo_1980_2024.sel(time=pdo_1980_2024.time.dt.month.isin([3, 4, 5]))
nino_34_1980_2024_mam = nino_34_1980_2024.sel(time=nino_34_1980_2024.time.dt.month.isin([3, 4, 5]))

In [ ]:
# Read and concatenate in order
storm_reports = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

In [ ]:
storm_reports['BEGIN_YEARMONTH'] = pd.to_datetime(storm_reports['BEGIN_YEARMONTH'], format='%Y%m')
storm_reports['END_YEARMONTH'] = pd.to_datetime(storm_reports['END_YEARMONTH'], format='%Y%m')

In [ ]:
# Extract the month
storm_reports["MONTH"] = storm_reports["BEGIN_YEARMONTH"].dt.month

In [ ]:
# Specify different storm types
tornados = storm_reports[storm_reports['EVENT_TYPE'].isin(['Tornado'])]
hail = storm_reports[storm_reports['EVENT_TYPE'].isin(['Hail'])]
tstorm_wind = storm_reports[storm_reports['EVENT_TYPE'].isin(['Thunderstorm Wind'])]

In [ ]:
# Subset storm reports by time periods
tornados_mam = tornados[tornados['BEGIN_YEARMONTH'].dt.month.isin([3, 4, 5])]
tornados_mar = tornados[tornados['BEGIN_YEARMONTH'].dt.month.isin([3])]
tornados_apr = tornados[tornados['BEGIN_YEARMONTH'].dt.month.isin([4])]
tornados_may = tornados[tornados['BEGIN_YEARMONTH'].dt.month.isin([5])]

hail_mam = hail[hail['BEGIN_YEARMONTH'].dt.month.isin([3, 4, 5])]
hail_mar = hail[hail['BEGIN_YEARMONTH'].dt.month.isin([3])]
hail_apr = hail[hail['BEGIN_YEARMONTH'].dt.month.isin([4])]
hail_may = hail[hail['BEGIN_YEARMONTH'].dt.month.isin([5])]

tstorm_wind_mam = tstorm_wind[tstorm_wind['BEGIN_YEARMONTH'].dt.month.isin([3, 4, 5])]
tstorm_wind_mar = tstorm_wind[tstorm_wind['BEGIN_YEARMONTH'].dt.month.isin([3])]
tstorm_wind_apr = tstorm_wind[tstorm_wind['BEGIN_YEARMONTH'].dt.month.isin([4])]
tstorm_wind_may = tstorm_wind[tstorm_wind['BEGIN_YEARMONTH'].dt.month.isin([5])]

In [ ]:
# Want to mess around with grid size a little bit to see if we can get a more coherent signal
lat_edges = np.append(era5_data_conus['latitude'].values-0.25, 23.5)
lon_edges = np.append(era5_data_conus['longitude'].values-0.25, 294)

lat_edges_0_5deg = lat_edges[::2]
lon_edges_0_5deg = lon_edges[::2]

In [ ]:
hail_counts_mam_separated = np.zeros((135, 50, 118))

for i in range(45):
    hail_yr_mar = hail_mar.loc[hail_mar['BEGIN_YEARMONTH'].dt.year == 1980 + i]
    hail_yr_apr = hail_apr.loc[hail_apr['BEGIN_YEARMONTH'].dt.year == 1980 + i]
    hail_yr_may = hail_may.loc[hail_may['BEGIN_YEARMONTH'].dt.year == 1980 + i]

    hail_counts_mam_separated[(i*3)] = pdo_functions.count_in_grid(hail_yr_mar['BEGIN_LAT'], pdo_functions.convert_longitudes(hail_yr_mar['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)
    hail_counts_mam_separated[(i*3)+1] = pdo_functions.count_in_grid(hail_yr_apr['BEGIN_LAT'], pdo_functions.convert_longitudes(hail_yr_apr['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)
    hail_counts_mam_separated[(i*3)+2] = pdo_functions.count_in_grid(hail_yr_may['BEGIN_LAT'], pdo_functions.convert_longitudes(hail_yr_may['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)

In [ ]:
tstorm_wind_counts_mam_separated = np.zeros((135, 50, 118))

for i in range(45):
    tstorm_wind_yr_mar = tstorm_wind_mar.loc[tstorm_wind_mar['BEGIN_YEARMONTH'].dt.year == 1980 + i]
    tstorm_wind_yr_apr = tstorm_wind_apr.loc[tstorm_wind_apr['BEGIN_YEARMONTH'].dt.year == 1980 + i]
    tstorm_wind_yr_may = tstorm_wind_may.loc[tstorm_wind_may['BEGIN_YEARMONTH'].dt.year == 1980 + i]

    tstorm_wind_counts_mam_separated[(i*3)] = pdo_functions.count_in_grid(tstorm_wind_yr_mar['BEGIN_LAT'], pdo_functions.convert_longitudes(tstorm_wind_yr_mar['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)
    tstorm_wind_counts_mam_separated[(i*3)+1] = pdo_functions.count_in_grid(tstorm_wind_yr_apr['BEGIN_LAT'], pdo_functions.convert_longitudes(tstorm_wind_yr_apr['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)
    tstorm_wind_counts_mam_separated[(i*3)+2] = pdo_functions.count_in_grid(tstorm_wind_yr_may['BEGIN_LAT'], pdo_functions.convert_longitudes(tstorm_wind_yr_may['BEGIN_LON']), np.sort(lat_edges_0_5deg), lon_edges_0_5deg)

In [ ]:
tstorm_wind_counts_mam_separated_mean = tstorm_wind_counts_mam_separated.mean(axis=0)
hail_counts_mam_separated_mean = hail_counts_mam_separated.mean(axis=0)
tornado_counts_mam_separated_mean = tornado_counts_mam_separated.mean(axis=0)

tstorm_wind_counts_mam_separated_mean[tstorm_wind_counts_mam_separated_mean == 0] = np.nan
hail_counts_mam_separated_mean[hail_counts_mam_separated_mean == 0] = np.nan
tornado_counts_mam_separated_mean[tornado_counts_mam_separated_mean == 0] = np.nan

In [ ]:
# Compositing
tstorm_wind_pos_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] > 0.5, tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean
tstorm_wind_neg_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] < -0.5, tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean

tornado_pos_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] > 0.5, tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean
tornado_neg_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] < -0.5, tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean

hail_pos_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] > 0.5, hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean
hail_neg_pdo = np.where(pdo_1980_2024_mam.values[:, None, None] < -0.5, hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean

tstorm_wind_pos_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] > 0.5, tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean
tstorm_wind_neg_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] < -0.5, tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean

tornado_pos_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] > 0.5, tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean
tornado_neg_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] < -0.5, tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean

hail_pos_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] > 0.5, hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean
hail_neg_nino = np.where(nino_34_1980_2024_mam.values[:, None, None] < -0.5, hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean

tstorm_wind_pos_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] > 0) & (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean
tstorm_wind_neg_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] < 0) & (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), tstorm_wind_counts_mam_separated, np.nan) - tstorm_wind_counts_mam_separated_mean

tornado_pos_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] > 0) & (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean
tornado_neg_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] < 0) & (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), tornado_counts_mam_separated, np.nan) - tornado_counts_mam_separated_mean

hail_pos_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] > 0) & (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean
hail_neg_pdo_neut_nino = np.where((pdo_1980_2024_mam.values[:, None, None] < 0)& (nino_34_1980_2024_mam.values[:, None, None] < 0.5) & (nino_34_1980_2024_mam.values[:, None, None] > -0.5), hail_counts_mam_separated, np.nan) - hail_counts_mam_separated_mean

In [ ]:
fig, ax = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2]]

tstorm_wind_bounds = [-1.4, -1.2, -1, -0.8, -0.6, -0.5, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05, -0.025, 0.025, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.2, 1.4]
tstorm_wind_norm = c.BoundaryNorm(tstorm_wind_bounds, plt.get_cmap('cmr.fusion_r').N)

tornado_bounds = [-0.35, -0.3, -0.25, -0.2, -0.15, -0.125, -0.1, -0.075, -0.05, -0.0375, -0.025, -0.0125, -0.00625, 0.00625, 0.0125, 0.025, 0.0375, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2, 0.25, 0.3, 0.35]
tornado_norm = c.BoundaryNorm(tornado_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(6):
    ax[i].coastlines(lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.25)
    ax[i].set_facecolor('xkcd:light gray')
    
ax[0].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_pos_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')
ax[3].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_neg_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')

ax[1].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_pos_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[4].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_neg_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[2].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_pos_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[5].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_neg_pdo, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[0].set_title('Tornados', fontsize=10, pad=4)
ax[1].set_title('Thunderstorm Wind', fontsize=10, pad=4)
ax[2].set_title('Hail', fontsize=10, pad=4)

ax[0].text(-3500000, -120000, '(+)', rotation='horizontal', fontweight='normal', fontsize=10)
ax[3].text(-3400000, -350000, '(-)', rotation='horizontal', fontweight='normal', fontsize=10)

cax = plt.axes([0.02, .22, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tornado_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', ticks=[-0.35, -0.15, 0, 0.15, 0.35])
cbar.set_label(r'tornado reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.5)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .22, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar2.set_label(r'thunderstorm wind reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.5)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.70, .22, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar3.set_label(r'hail reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.5)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.75, 
                    wspace=0.05, 
                    hspace=0.05)

In [ ]:
fig, ax = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2]]

tstorm_wind_bounds = [-1.4, -1.2, -1, -0.8, -0.6, -0.5, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05, -0.025, 0.025, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.2, 1.4]
tstorm_wind_norm = c.BoundaryNorm(tstorm_wind_bounds, plt.get_cmap('cmr.fusion_r').N)

tornado_bounds = [-0.35, -0.3, -0.25, -0.2, -0.15, -0.125, -0.1, -0.075, -0.05, -0.0375, -0.025, -0.0125, -0.00625, 0.00625, 0.0125, 0.025, 0.0375, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2, 0.25, 0.3, 0.35]
tornado_norm = c.BoundaryNorm(tornado_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(6):
    ax[i].coastlines(lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.25)
    ax[i].set_facecolor('xkcd:light gray')
    
ax[0].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_pos_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')
ax[3].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_neg_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')

ax[1].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_pos_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[4].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_neg_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[2].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_pos_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[5].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_neg_pdo_neut_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[0].set_title('Tornados', fontsize=10, pad=4)
ax[1].set_title('Thunderstorm Wind', fontsize=10, pad=4)
ax[2].set_title('Hail', fontsize=10, pad=4)

ax[0].text(-3500000, -120000, '(+)', rotation='horizontal', fontweight='normal', fontsize=10)
ax[3].text(-3400000, -350000, '(-)', rotation='horizontal', fontweight='normal', fontsize=10)

cax = plt.axes([0.02, .22, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tornado_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', ticks=[-0.35, -0.15, 0, 0.15, 0.35])
cbar.set_label(r'tornado reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.5)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .22, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar2.set_label(r'thunderstorm wind reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.5)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.70, .22, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar3.set_label(r'hail reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.5)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.75, 
                    wspace=0.05, 
                    hspace=0.05)

In [ ]:
fig, ax = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], ax[0, 2], \
      ax[1, 0], ax[1, 1], ax[1, 2]]

tstorm_wind_bounds = [-1.4, -1.2, -1, -0.8, -0.6, -0.5, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05, -0.025, 0.025, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1, 1.2, 1.4]
tstorm_wind_norm = c.BoundaryNorm(tstorm_wind_bounds, plt.get_cmap('cmr.fusion_r').N)

tornado_bounds = [-0.35, -0.3, -0.25, -0.2, -0.15, -0.125, -0.1, -0.075, -0.05, -0.0375, -0.025, -0.0125, -0.00625, 0.00625, 0.0125, 0.025, 0.0375, 0.05, 0.075, 0.1, 0.125, 0.15, 0.2, 0.25, 0.3, 0.35]
tornado_norm = c.BoundaryNorm(tornado_bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(6):
    ax[i].coastlines(lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.25)
    ax[i].set_facecolor('xkcd:light gray')
    
ax[0].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_pos_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')
ax[3].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tornado_neg_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tornado_norm, cmap='cmr.fusion_r')

ax[1].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_pos_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[4].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(tstorm_wind_neg_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[2].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_pos_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')
ax[5].pcolormesh(lon_edges_0_5deg, lat_edges_0_5deg, np.flipud(np.nanmean(hail_neg_nino, axis=0)), transform=ccrs.PlateCarree(), norm=tstorm_wind_norm, cmap='cmr.fusion_r')

ax[0].set_title('Tornados', fontsize=10, pad=4)
ax[1].set_title('Thunderstorm Wind', fontsize=10, pad=4)
ax[2].set_title('Hail', fontsize=10, pad=4)

ax[0].text(-3500000, -120000, '(+)', rotation='horizontal', fontweight='normal', fontsize=10)
ax[3].text(-3400000, -350000, '(-)', rotation='horizontal', fontweight='normal', fontsize=10)

cax = plt.axes([0.02, .22, 0.29, 0.02])
cbar = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tornado_norm), cax=cax, orientation='horizontal', spacing='proportional', extend='both', ticks=[-0.35, -0.15, 0, 0.15, 0.35])
cbar.set_label(r'tornado reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar.outline.set_linewidth(0.5)
cbar.outline.set_color('black')
# cbar.outline.set_visible(False)

cax2 = plt.axes([0.355, .22, 0.29, 0.02])
cbar2 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax2, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar2.set_label(r'thunderstorm wind reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar2.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar2.outline.set_linewidth(0.5)
cbar2.outline.set_color('black')
# cbar.outline.set_visible(False)

cax3 = plt.axes([0.70, .22, 0.29, 0.02])
cbar3 = plt.colorbar(mpl.cm.ScalarMappable(cmap='cmr.fusion_r', norm=tstorm_wind_norm), cax=cax3, orientation='horizontal', spacing='proportional', extend='both', ticks=[-1.4, -0.6, 0, 0.6, 1.4])
cbar3.set_label(r'hail reports month$^{-1}$', size=8, fontweight='normal', color='black')
cbar3.ax.tick_params(which='both', labelsize=8, width=0.5, length=0, labelcolor='black')
cbar3.outline.set_linewidth(0.5)
cbar3.outline.set_color('black')
# cbar.outline.set_visible(False)

plt.subplots_adjust(left=0,
                    bottom=0.25, 
                    right=1., 
                    top=0.75, 
                    wspace=0.05, 
                    hspace=0.05)